In [2]:
!pip install ultralytics torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 56.0 MB/s eta 0:00:00


In [3]:
import ultralytics, torch, torchvision, torch.nn as nn
from ultralytics import YOLO
import torch.nn.functional as F

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [14]:
!unzip /content/standard.zip

Streaming output truncated to the last 5000 lines.
  inflating: standard/train/labels/dtundra04241_png_jpg.rf.319c1e0e68cf620d4a5fc2336d265f9e.txt  
  inflating: standard/train/labels/dtacoma03301_png_jpg.rf.ba2aca3a967fd2badc5b769798562762.txt  
  inflating: standard/train/labels/detect01000_png_jpg.rf.599d58e35ad9fd536814842a3e8a829c.txt  
  inflating: standard/train/labels/drench00609_png_jpg.rf.930d9345a51c22bf58fd26339e7cf1ca.txt  
  inflating: standard/train/labels/dry00402_png_jpg.rf.c09c373d0a088d2ae24d906b8326a7e0.txt  
  inflating: standard/train/labels/detect00808_png_jpg.rf.9d4f339a90bcb6248b7cb36596d11627.txt  
  inflating: standard/train/labels/dirt00458_png_jpg.rf.a1d50c82949f752ec2b9a574bdfad8f2.txt  
  inflating: standard/train/labels/down00983_png_jpg.rf.74055c8e88d05f2de1cbde565567d7c5.txt  
  inflating: standard/train/labels/dvibe02151_png_jpg.rf.6077a86088b7fe98c959bd09593fa4d8.txt  
  inflating: standard/train/labels/damage00548_png_jpg.rf.f37e320e68fc5e9cc2785b49

In [10]:
class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction_ratio=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        # Adjusted reduction ratio for v11m's larger channel counts
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(channels // reduction_ratio, channels)
        )

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x).view(x.size(0), -1))
        max_out = self.fc(self.max_pool(x).view(x.size(0), -1))
        out = avg_out + max_out
        return torch.sigmoid(out).view(x.size(0), x.size(1), 1, 1)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=kernel_size//2)

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        return torch.sigmoid(self.conv(x))

class CBAM(nn.Module):
    def __init__(self, channels, reduction_ratio=16, spatial_kernel_size=7):
        super().__init__()
        self.channel_att = ChannelAttention(channels, reduction_ratio)
        self.spatial_att = SpatialAttention(spatial_kernel_size)

    def forward(self, x):
        x = x * self.channel_att(x)
        x = x * self.spatial_att(x)
        return x

In [11]:
class TransformerBlock(nn.Module):
    def __init__(self, dim, num_heads=8, mlp_ratio=4):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(dim, num_heads)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, dim * mlp_ratio),
            nn.GELU(),
            nn.Linear(dim * mlp_ratio, dim)
        )

    def forward(self, x):
        B, C, H, W = x.shape
        x = x.flatten(2).permute(2, 0, 1)  # (H*W, B, C)

        attn_out = self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        x = x + attn_out

        x = x + self.mlp(self.norm2(x))
        x = x.permute(1, 2, 0).view(B, C, H, W)
        return x

In [12]:
class YOLOWithAttention(nn.Module):
      def __init__(self, model_path='/content/yolo11l.pt'):
        super().__init__()
        self.yolo = YOLO(model_path)

        # Get the backbone and head modules for v11m
        self.backbone = self.yolo.model.model[:-3]
        self.head = self.yolo.model.model[-3:]

        # Adjusted channel dimensions for v11m
        self.cbam1 = CBAM(channels=512)    # After early stage
        self.cbam2 = CBAM(channels=1024)   # After middle stage
        self.transformer = TransformerBlock(dim=2048, num_heads=16)  # Before head

        # Optional: Add attention to detection heads
        self.head_attention = nn.ModuleList([
            CBAM(channels=ch) for ch in [512, 1024, 2048]  # Adjust based on your head dimensions
        ])

      def forward(self, x):
        features = []

        # Backbone forward pass with attention
        for i, layer in enumerate(self.backbone):
            x = layer(x)

            # Add attention at specific stages (adjusted for v11m architecture)
            if i == 6:  # Early stage
                x = self.cbam1(x)
            elif i == 8:  # Middle stage
                x = self.cbam2(x)
            elif i == 10:  # Before head
                x = self.transformer(x)

            features.append(x)

        # Head forward pass with additional attention
        head_outputs = []
        for i, layer in enumerate(self.head):
            x = layer(x)
            if i < len(self.head_attention):  # Apply attention to each head output
                x = self.head_attention[i](x)
            head_outputs.append(x)

        return head_outputs

In [ ]:
def train_yolo_with_attention():
    # Initialize model with v11m
    model = YOLOWithAttention('/content/yolo11l.pt')

    # Training configuration optimized for v11m
    model.yolo.train(
        data='/content/standard/dataset.yaml',
        epochs=300,
        imgsz=640,
        batch=32,  # Adjusted for v11m's larger memory requirements
        device='cuda:0',
        optimizer='AdamW',  # Better optimizer for attention mechanisms
        lr0=0.001,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3,
        warmup_momentum=0.8,
        warmup_bias_lr=0.1,
        box=7.5,
        cls=0.5,
        dfl=1.5,
        pose=12.0,
        kobj=1.0,
        label_smoothing=0.0,
        nbs=64,
        overlap_mask=True,
        mask_ratio=4,
        dropout=0.0,
        val=True
    )

In [ ]:
detect1 = train_yolo_with_attention()

WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.51 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/content/yolo11l.pt, data=/content/standard/dataset.yaml, epochs=300, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

100%|██████████| 755k/755k [00:00<00:00, 134MB/s]


Overriding model.yaml nc=80 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  2    173824  ultralytics.nn.modules.block.C3k2            [128, 256, 2, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  2    691712  ultralytics.nn.modules.block.C3k2            [256, 512, 2, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2234368  ultralytics.nn.modules.block.C3k2            [512, 512, 2, True]           
  7                  -1  1   2360320  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 366MB/s]


AMP: checks passed ✅


...

     43/300        21G      1.554      1.197      1.465        171        640: 100%|██████████| 180/180 [02:47<00:00,  1.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:05<00:00,  2.10it/s]

                   all        720       2435      0.653      0.591      0.622      0.311



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     44/300      21.1G      1.543       1.17       1.45        166        640:  25%|██▌       | 45/180 [00:42<02:06,  1.06it/s]


KeyboardInterrupt: 

In [30]:
import locale
import os

# Set locale to UTF-8
os.environ['LC_ALL'] = 'en_US.UTF-8'
os.environ['LANG'] = 'en_US.UTF-8'
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [31]:
print(locale.getdefaultlocale())

('en_US', 'UTF-8')


In [ ]:
!zip -r /content/runs.zip /content/runs

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [5]:
def train_yolo_with_attention():
    # Initialize model with v11m
    model = YOLOWithAttention('/content/drive/MyDrive/defect/train3_modified.pt')

    # Training configuration optimized for v11m
    model.yolo.train(
        data='/content/standard/dataset.yaml',
        epochs=300,
        imgsz=640,
        batch=32,  # Adjusted for v11m's larger memory requirements
        device='cuda:0',
        optimizer='AdamW',  # Better optimizer for attention mechanisms
        lr0=0.001,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3,
        warmup_momentum=0.8,
        warmup_bias_lr=0.1,
        box=7.5,
        cls=0.5,
        dfl=1.5,
        pose=12.0,
        kobj=1.0,
        label_smoothing=0.0,
        nbs=64,
        overlap_mask=True,
        mask_ratio=4,
        dropout=0.0,
        val=True,
        resume=True
    )

In [ ]:
detect1 = train_yolo_with_attention()

WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.51 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/content/runs/detect/train/weights/last.pt, data=/content/standard/dataset.yaml, epochs=300, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=/content/runs/detect/train/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None

train: Scanning /content/standard/train/labels.cache... 5758 images, 3 backgrounds, 0 corrupt: 100%|██████████| 5758/5758 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/standard/valid/labels.cache... 720 images, 0 backgrounds, 0 corrupt: 100%|██████████| 720/720 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
Resuming training /content/runs/detect/train/weights/last.pt from epoch 44 to 300 total epochs
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    153/300      21.2G      1.039     0.6309      1.119        179        640: 100%|██████████| 180/180 [02:47<00:00,  1.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:05<00:00,  2.13it/s]

                   all        720       2435      0.722      0.661      0.681      0.344



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    154/300      21.1G      1.059     0.6552       1.13        200        640:  21%|██        | 38/180 [00:35<02:11,  1.08it/s]

---
**Hope that it is not all lost**
---

In [32]:
def train_yolo_with_attention():
    # Initialize model with v11m
    model = YOLOWithAttention('/content/drive/MyDrive/defect/train3_modified.pt')

    # Training configuration optimized for v11m
    model.yolo.train(
        data='/content/standard/dataset.yaml',
        epochs=50,
        imgsz=640,
        batch=32,  # Adjusted for v11m's larger memory requirements
        device='cuda:0',
        optimizer='AdamW',  # Better optimizer for attention mechanisms
        lr0=0.00005,
        lrf=0.0005,
        momentum=0.937,
        weight_decay=0.0007,
        warmup_epochs=5,
        warmup_momentum=0.8,
        warmup_bias_lr=0.1,
        box=8.0,
        cls=0.4,
        dfl=1.5,
        pose=12.0,
        kobj=1.0,
        label_smoothing=0.0,
        nbs=64,
        overlap_mask=True,
        mask_ratio=4,
        dropout=0.0,
        val=True,
        resume=True
    )

In [33]:
test_model = train_yolo_with_attention()

WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.58 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/defect/train3_modified.pt, data=/content/standard/dataset.yaml, epochs=300, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=2, project=None, name=train645, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=/content/drive/MyDrive/defect/train3_modified.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False

train: Scanning /content/standard/train/labels.cache... 5758 images, 3 backgrounds, 0 corrupt: 100%|██████████| 5758/5758 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/standard/valid/labels.cache... 720 images, 0 backgrounds, 0 corrupt: 100%|██████████| 720/720 [00:00<?, ?it/s]


Plotting labels to runs/detect/train645/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Resuming training /content/drive/MyDrive/defect/train3_modified.pt from epoch 302 to 300 total epochs
DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv):

...

    312/600      16.6G       1.43      1.052      1.303        186        640: 100%|██████████| 180/180 [02:11<00:00,  1.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:04<00:00,  2.44it/s]

                   all        720       2435      0.682      0.593      0.628      0.303



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    313/600      16.7G       1.42      1.042      1.291        253        640:  47%|████▋     | 85/180 [01:02<01:09,  1.36it/s]


KeyboardInterrupt: 

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968